In [ ]:
import os
import json,math
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as clr
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_context('talk')
import warnings
warnings.filterwarnings('ignore')
idx = pd.IndexSlice
pj = os.path.join

In [ ]:
root_dir = ".."
data_surf_dir = pj(root_dir, 'data', "surface_counts")
data_plot_dir = pj(root_dir, 'data', "for_plots")
fig_dir = "panels_sup2"

do_save_plots = False

## pMHC loading calibration numbers

In [ ]:
#@title Tools
from scipy.optimize import curve_fit, lsq_linear
from scipy.stats import linregress

def michaelis_menten(x, a, h):
    normx = (x / h)
    return a * normx / (1.0 + normx)

def loglog_michaelis_menten(logx, a, h):
    logh = np.log(h)
    return np.log(a * np.exp(logx - logh) / (1.0 + np.exp(logx - logh)))

def loglog_michaelis_background(logx, a, h, back):
    logh = np.log(h)
    # a is the real amplitude, there is a background on top, additive.
    return np.log(a*np.exp(logx - logh) / (1.0 + np.exp(logx - logh)) + back)

def bootstrap_f_ci(x, f, a=0.05, reps=100, rel_size=0.8, rgen=None):
    # Generate reps bootstrap samples, compute f of each, compute 95 % CI of f
    # f could be geometric mean for instance.
    if rgen is None:
        rgen = np.random.default_rng()
    boot_f_samples = []
    for i in range(reps):
        bootsamp = rgen.choice(x, size=int(rel_size*x.size), replace=True)
        boot_f_samples.append(f(bootsamp))
    # Confidence intervals
    boot_f_samples = np.asarray(boot_f_samples)
    ci_bounds = np.quantile(boot_f_samples, [a/2, 1.0-a/2])
    labels = ["CI "+str(round(x, 3)) for x in [a/2, 1.0-a/2]]
    return {labels[i]:ci_bounds[i] for i in range(2)}

def geo_mean(x):
    return np.exp(np.log(x).mean())

In [ ]:
#@title Load raw summarized data
df_rmas = pd.read_hdf(pj(data_surf_dir, "combinedRMASDf_MFI.hdf"), key="df")
# Convert concentrations to uM, more convenient to read.
df_rmas = df_rmas.rename(lambda x: x*1e6, level="Concentration_M")
# Reorder levels to more convenient form
df_rmas = df_rmas.reorder_levels(["Experiment", "Replicate", "Peptide", "Concentration_M"])
df_rmas.index = df_rmas.index.set_names(["Experiment", "Replicate", "Peptide", "Concentration_uM"])
df_rmas

For every peptide, every replicate, fit Michaelis-Menten $R = R_0 L / (L + K_D)$, but fit in log-log scale: if $y=\log(R)$ and $x=\log(L)$, fit $y$ vs $x$ in the following relationship: $y = \log\left(\frac{R_0  e^x}{e^x + K_D} \right)$.
So MM parameters describe relationship in linear scale, we just fitted them to minimize error on log-scale to ensure multiple orders of magnitude are well captured. This is a linear MM relationship plotted in log-log, not a MM function fitted on the log-log plot.

In [ ]:
#@title Define fitting function
def fit_ligand_receptor_mm(df):
    """ To be applied on the sub-dataframe corresponding to one dose-response. """
    x = np.log(df.index.get_level_values("Concentration_uM").values)
    y = df.values.flatten()
    pbounds = np.asarray([(0.7*np.amax(y), np.exp(x.min())/10.0, 0.5*np.amin(y)),
               (2.0*np.amax(y), np.exp(x.max())*10, 1.5*np.amin(y))])
    y = np.log(y)
    l_conc_mm_params = curve_fit(loglog_michaelis_background, x, y, p0=np.mean(pbounds, axis=0), bounds=pbounds)[0]
    return l_conc_mm_params

In [ ]:
#@title Fit Michaelis-Menten curves
all_params = []
gby = df_rmas.groupby(["Experiment", "Replicate", "Peptide"])
for lbl, gp in gby:
    fit_params = fit_ligand_receptor_mm(gp)
    all_params.append(fit_params)
all_params = pd.DataFrame(all_params, index=pd.MultiIndex.from_tuples(gby.groups,
                names=["Experiment", "Replicate", "Peptide"]),
                columns=["amplitude", "ec50", "background"])
all_params = all_params.reorder_levels(["Experiment", "Replicate", "Peptide"])
all_params

In [ ]:
#@title Plot fits to make sure they are reasonable
plot_idx = all_params.index.droplevel(["Peptide"]).unique()
n_plots = len(plot_idx)
fig, axes = plt.subplots(1, n_plots)
fig.set_size_inches(11, 4.5)
axes = axes.flatten()
all_params = all_params.sort_index()
df_rmas = df_rmas.sort_index()
df_curves = {}
peptides = list(all_params.index.get_level_values("Peptide").unique())
colors = sns.color_palette(n_colors=len(peptides))
pep_color_order = ["N4", "Q4", "T4", "V4", "G4", "E1", "A2", "Y3", "Q7"]
pep_palette = {p:colors[pep_color_order.index(p)] for p in peptides}
experiments = list(all_params.index.get_level_values("Experiment").unique())
peptides.remove("Y3")
for i, k in enumerate(plot_idx):
    for pep in all_params.loc[k].index.unique():
        if pep == "Y3": continue
        x = df_rmas.loc[k+(pep,)].index.get_level_values("Concentration_uM").values
        xrange = np.geomspace(x.min(), x.max(), 101)
        y = df_rmas.loc[k+(pep,)].values.flatten()
        pms = all_params.loc[k+(pep,)]
        ymod = np.exp(loglog_michaelis_background(np.log(xrange), *pms))
        xy_df = pd.DataFrame(np.stack([xrange, ymod], axis=1),
                             index=pd.RangeIndex(101, name="Point"),
                             columns=["x", "y"])
        df_curves[tuple(k) + (pep,)] = xy_df.copy()
        c = pep_palette.get(pep)
        axes[i].plot(xrange, ymod, label=pep, color=c)
        axes[i].plot(x, y, color=c, mfc=c, mec=c, marker="o", ms=6, ls="none")
        axes[i].axvline(pms[1], ls="--", lw=1.5, color=c)
    i_exp = experiments.index(k[0])
    axes[i].set_title("Exp. {}, replicate {}".format(i_exp+1, k[1]))
    axes[i].set(xlabel=r"Pulse concentration ($\mu$M)", yscale="log", xscale="log")
    for pos in ["top", "right"]:
        axes[i].spines[pos].set_visible(False)
axes[0].set_ylabel("Mean fluorescence (a.u.)")
df_curves = pd.concat(df_curves, names=all_params.index.names)
df_curves

fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(fig_dir, "rmas_mfi_fits_full.pdf"), transparent=True,
            bbox_inches="tight", bbox_extra_artists=(axes[-1].xaxis.label,))
plt.show()
plt.close()

In [ ]:
#@title Version showing one representative replicate
# Dropping Y3 too
plot_idx = all_params.index.droplevel(["Peptide"]).unique()
k_exp = plot_idx[0]
fig, ax = plt.subplots()
figsize = fig.get_size_inches()
fig.set_size_inches(figsize[0]*.825, 4.5)

all_params = all_params.sort_index()
df_rmas = df_rmas.sort_index()
peptides = list(all_params.index.get_level_values("Peptide").unique())
colors = sns.color_palette(n_colors=len(peptides))
pep_color_order = ["N4", "Q4", "T4", "V4", "G4", "E1", "A2", "Y3", "Q7"]
pep_palette = {p:colors[pep_color_order.index(p)] for p in peptides}
for pep in all_params.loc[k_exp].index.unique():
    if pep == "Y3": continue
    x = df_rmas.loc[k_exp+(pep,)].index.get_level_values("Concentration_uM").values
    xrange = np.geomspace(x.min(), x.max(), 101)
    y = df_rmas.loc[k_exp+(pep,)].values.flatten()
    pms = all_params.loc[k_exp+(pep,)]
    ymod = np.exp(loglog_michaelis_background(np.log(xrange), *pms))
    c = pep_palette.get(pep)
    ax.plot(xrange, ymod, label=pep, color=c)
    ax.plot(x, y, color=c, mfc=c, mec=c, marker="o", ms=6, ls="none")
    #ax.axvline(pms[1], ls="--", lw=1.5, color=c)
    ax.plot([pms[1]]*2, [-np.inf, 0.5*pms[0] + pms[2]], ls="--", lw=1.5, color=c)
    i_exp = experiments.index(k_exp[0])
    #ax.set_title("Exp. {}, replicate {}".format(i_exp+1, k_exp[1]))
    ax.set(xlabel=r"Pulse concentration ($\mu$M)", yscale="log", xscale="log")
    for pos in ["top", "right"]:
        ax.spines[pos].set_visible(False)
ax.set_ylabel("H-2kb fluorescence (a.u.)")

fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(fig_dir, "rmas_mfi_fits.pdf"), transparent=True,
            bbox_inches="tight", bbox_extra_artists=(axes[-1].xaxis.label,))
plt.show()
plt.close()

In [ ]:
#@title Statistics of loading EC50s
randomgen = np.random.default_rng(0x339788df8e03ab6f23168dec3dd313f3)
booted_params = np.log(all_params.T.copy())
booted_params = booted_params.apply(bootstrap_f_ci, axis=1, raw=False, args=(np.mean,),
                        a=0.05, reps=1000, rel_size=0.8, rgen=randomgen, result_type="expand")
# Add mean
booted_params = np.exp(booted_params)
booted_params["Geometric mean"] = all_params.T.apply(geo_mean, axis=1)

# We only need the K_D; amplitude and background are relative to specific MFI here
# And will be replaced by the number of MHC on each APC type.
mhc_pulse_kd = booted_params.loc["ec50"]
mhc_pulse_kd.name = "K_D (uM)"

# Add 95 % CI of the distribution (not just CI of mean)
booted_lower_ci = np.log(all_params.T).apply(bootstrap_f_ci, axis=1, raw=False,
                                             args=(lambda x: np.quantile(x, 0.025),),
                        a=0.05, reps=1000, rel_size=0.8, rgen=randomgen, result_type="expand")
booted_upper_ci = np.log(all_params.T).apply(bootstrap_f_ci, axis=1, raw=False,
                                             args=(lambda x: np.quantile(x, 0.975),),
                        a=0.05, reps=1000, rel_size=0.8, rgen=randomgen, result_type="expand")
mhc_pulse_kd.loc["Quantile 0.025"] = np.exp(booted_lower_ci.loc["ec50"].mean())
mhc_pulse_kd.loc["Quantile 0.975"] = np.exp(booted_upper_ci.loc["ec50"].mean())
mhc_pulse_kd  # in uM

In [ ]:
#@title Summary of loading EC50s
fig, ax = plt.subplots()
figsize = fig.get_size_inches()
fig.set_size_inches(figsize[0]*.8, 4.5)
pep_affinity_order = ["N4", "A2", "Y3", "Q4", "T4", "Q7", "V4", "G4", "E1"]
peptides_ordered = [p for p in pep_affinity_order if p in peptides]
peptides_ordered.remove("Y3")
for i, pep in enumerate(peptides_ordered):
    ec50s_pep = all_params["ec50"].xs(pep, level="Peptide")
    jitter = 0.1*np.random.normal(size=ec50s_pep.size)
    c = pep_palette.get(pep)
    ax.plot(i+jitter, ec50s_pep, mfc=c, mec=c, marker="o", ms=6,
            ls="none", label=pep)
    # Only plot statistics
    ec50_pep_mean = geo_mean(ec50s_pep)
    ec50_pep_log_std = np.std(np.log(ec50s_pep))
    mhc_kd_errs = [
         [ec50_pep_mean - np.exp(np.log(ec50_pep_mean) - ec50_pep_log_std)],
         [np.exp(np.log(ec50_pep_mean) + ec50_pep_log_std) - ec50_pep_mean]]
    #ax.errorbar(i, ec50_pep_mean, yerr=mhc_kd_errs, mfc=c, mec=c,
    #            marker="o", ms=6, ls="none")

# Also annotate the geometric mean and CI.
mhc_kd_errs = [[mhc_pulse_kd["Geometric mean"] - mhc_pulse_kd["CI 0.025"]],
               [mhc_pulse_kd["CI 0.975"] - mhc_pulse_kd["Geometric mean"]]]
ax.errorbar(len(peptides_ordered)+0.5, mhc_pulse_kd["Geometric mean"], yerr=mhc_kd_errs,
         color="k", marker="s", ms=8, capsize=2, capthick=2)
xlims = ax.get_xlim()
ax.fill_between(x=xlims, y1=mhc_pulse_kd["Quantile 0.025"], y2=mhc_pulse_kd["Quantile 0.975"],
                color="k", alpha=0.15)
ax.axhline(mhc_pulse_kd["Geometric mean"], ls="--", color="k")
ax.set_xlim(xlims)
ax.set_xticks(list(range(len(peptides_ordered))) + [len(peptides_ordered)+0.5])
ax.set_xticklabels(peptides_ordered + ["Geo.\nmean"])
ax.tick_params(axis="x", labelsize="small")
for pos in ["top", "right"]:
    ax.spines[pos].set_visible(False)
ax.set(ylabel=r"Loading EC$_{50}$ ($\mu$M)", yscale="log")
ax.set_xlabel("Peptide", labelpad=0.1)
#ax.legend(loc="upper left", bbox_to_anchor=(0.9, 1.0), frameon=False, title="OVA Peptide")
ax.set_ylim([1e-2, 1e0])
#ticks = ax.get_xticklabels()
#ticks[-1].set_rotation(30)
fig.tight_layout()

if do_save_plots:
    fig.savefig(pj(fig_dir, "rmas_loading_ec50s.pdf"), transparent=True, bbox_inches="tight")

plt.show()
plt.close()

In [ ]:
#@title Fraction of loaded pMHC
# Instead of showing absolute pMHC numbers for some cell type,
# show fraction loaded MHC, applicable to any cell type having some total MHC number
# setting the amplitude of that curve.
ag_pulse_range = np.geomspace(df_rmas.index.get_level_values("Concentration_uM").values.min(),
                             df_rmas.index.get_level_values("Concentration_uM").values.max(), 201)
fraction_loaded_mhc = michaelis_menten(ag_pulse_range, 1.0, mhc_pulse_kd["Geometric mean"])

# Upper and lower CI as well
# Make K_D as small as possible and amplitude as large as possible
upper_ci_mhc = michaelis_menten(ag_pulse_range, 1.0, mhc_pulse_kd["Quantile 0.025"])
lower_ci_mhc = michaelis_menten(ag_pulse_range, 1.0, mhc_pulse_kd["Quantile 0.975"])

fig, ax = plt.subplots()
figsize = fig.get_size_inches()
fig.set_size_inches(figsize[0]*0.825, 4.5)
fill_kwargs = dict(color="k", alpha=0.15)
ax.fill_between(ag_pulse_range, lower_ci_mhc, upper_ci_mhc, **fill_kwargs)
ax.plot(ag_pulse_range, fraction_loaded_mhc, color="k")
for pos in ["top", "right"]:
    ax.spines[pos].set_visible(False)
ax.set(xscale="log", yscale="log", xlabel=r"Pulse concentration ($\mu$M)",
       ylabel="Fraction loaded MHC")

ylims = ax.get_ylim()
ax.plot([mhc_pulse_kd["Geometric mean"],]*2, [ylims[0], 0.5], ls="--",
        color="k", label="Loading\n" + r"EC$_{50}$")
x_fill = np.linspace(mhc_pulse_kd["Quantile 0.025"], mhc_pulse_kd["Quantile 0.975"], 21)
y_fill_upper = michaelis_menten(x_fill, 1.0, mhc_pulse_kd["Geometric mean"])
ax.fill_between(x_fill, y_fill_upper, label="95 % CI", **fill_kwargs)
ax.set_ylim(ylims)
ax.legend(frameon=False, fontsize="small", loc="upper left")

fig.tight_layout()

if do_save_plots:
    fig.savefig(pj(fig_dir, "fraction_mhc_loaded.pdf"), transparent=True, bbox_inches="tight")

plt.show()
plt.close()


## Antigen EC50 to $\tau$ conversion

In [ ]:
#@title Tools
def standalone_legend(*leg_args, **leg_kwargs):
    fig1, ax1 = plt.subplots()
    ax1.set_axis_off()
    leg1 = ax1.legend(*leg_args, **leg_kwargs)
    fig1.canvas.draw()
    # First dummy drawing to get the legend size in inches
    leg_width = leg1.get_window_extent().width / fig1.dpi
    leg_height = leg1.get_window_extent().height / fig1.dpi
    plt.close()

    # Now, actual figure which we set at the right size
    fig, ax = plt.subplots()
    fig.set_size_inches(leg_width*1.05, leg_height*1.05)
    ax.set_axis_off()
    leg = ax.legend(*leg_args, **leg_kwargs)
    fig.tight_layout()
    return fig, ax, leg

# Version based on a relative EC50 and a reference tau
def convert_ec50_tau_relative(ec50rel, reftau, npow):
    """ Compute tau associated to an EC50, given a reference tau
    and the EC50 relative to that tau (EC50 / EC50ref). """
    return reftau / ec50rel**(1.0 / npow)

# Given relative ec50
def plot_ec50_tau_conversion(rel_ec50s, tau_ref, npow):
    ec50_range = np.geomspace(rel_ec50s.min()*0.5, rel_ec50s.max()*2, 150)
    # Model curve
    tau_range = convert_ec50_tau_relative(ec50_range, tau_ref, npow)
    # Peptide points
    pep_taus, pep_ec50s = [], []
    for pep in rel_ec50s.index:
        pep_taus.append(convert_ec50_tau_relative(rel_ec50s[pep], tau_ref, npow))
        pep_ec50s.append(rel_ec50s[pep])
    # Palette
    peptides = list(rel_ec50s.index)
    pep_color_order = ["N4", "Q4", "T4", "V4", "G4", "E1", "A2", "Y3", "Q7"]
    colors = sns.color_palette(n_colors=len(pep_color_order))
    pep_palette = {p:colors[pep_color_order.index(p)] for p in peptides}

    # Plot
    fig, ax = plt.subplots()
    figsize = fig.get_size_inches()
    fig.set_size_inches(figsize[0]*0.825, 4.5)
    ax.plot(ec50_range, tau_range, color="k", ls="-")
    for i, pep in enumerate(peptides):
        ax.plot(pep_ec50s[i], pep_taus[i], label=pep, marker="o", ls="none",
                color=pep_palette[pep])
    # Mark reference point
    ylims = ax.get_ylim()
    yfrac = (tau_ref-ylims[0]) / np.diff(ylims)[0]
    ax.axvline(1.0, ymin=0, ymax=yfrac,
               ls="--", color="grey", zorder=-10)
    ax.set(xscale="log", xlabel=r"Relative $\mathrm{EC_{50}}$", ylabel=r"TCR antigen $\tau^T$ (s)")
    xlims = ax.get_xlim()
    xfrac = (np.log(1.0) - np.log(xlims[0])) / np.diff(np.log(xlims))[0]
    ax.axhline(tau_ref, xmin=0, xmax=xfrac,
               ls="--", color="grey", zorder=-10)
    ax.annotate(r"$\tau \sim \mathrm{EC_{50}}^{-1/" + str(npow) + "}$", xy=(2.0, pep_taus[1]-1.5),
        xycoords="data", ha="left", va="top", rotation=-45)
    for side in ["top", "right"]:
        ax.spines[side].set_visible(False)
    #n_leg_col = len(pep_taus) // 5 + min(1, len(pep_taus) % 5)
    #ax.legend(ncol=n_leg_col, handlelength=2.0, columnspacing=0.3,
    #          handletextpad=0.3, frameon=False, labelspacing=0.3)

    leg_handles_labels = ax.get_legend_handles_labels()
    fig.tight_layout()
    return fig, ax, leg_handles_labels


In [ ]:
#@title Conversion plot
# Load EC50
ec50data = pd.read_json(pj(root_dir, "data", "dose_response", "potencies_df_2021.json"))
ec50data = np.exp(np.log(ec50data).mean(axis=1))
ec50data = ec50data.drop(["Y3", "Q7"])
with open(pj(root_dir, "data", "reference_pep_tau_maps.json"), "r") as h:
    ref_file = json.load(h)

# First plot with E1 to generate handles, labels for legends
fig, ax, handles_labels = plot_ec50_tau_conversion(ec50data, ref_file["N4"], 6)
plt.close()

# Standalone legend with E1, one column
figl, axl, _ = standalone_legend(*handles_labels, frameon=False, handlelength=2.0, columnspacing=0.3,
                                 handletextpad=0.3,  labelspacing=0.3, ncol=2, title="OVA Peptide")
if do_save_plots:
    figl.savefig(pj(fig_dir, "tau_ec50_conversion_legend.pdf"), transparent=True, bbox_inches="tight")

# New plot without E1 on it, because E1 was determined separately
ec50data = ec50data.drop("E1")
fig, ax, handles_labels = plot_ec50_tau_conversion(ec50data, ref_file["N4"], 6)
if do_save_plots:
    fig.savefig(pj(fig_dir, "tau_ec50_conversion.pdf"), transparent=True, bbox_inches="tight")

plt.show()
plt.close()

# Bar plots for surface molecule numbers

In [ ]:
#@title Load data
completeMoleculeNumberDf = pd.read_hdf(pj(data_surf_dir, 'completeMoleculeNumberDf.hdf'),key='df')
completeMoleculeNumberDf

In [ ]:
#@title Aesthetics
globalAspect =0.8

In [ ]:
#@title Choose cell names that appear in legends
cellTypeRenameDict = {
    "B6":"B6 Splenocyte",
    "B16": "B16 Melanoma",
    "OT1_Blast": "OT1 Blast",
    "OT1_Naive": "OT1 Naive",
    "OT1_CAR": "OT1 CAR",
    "E2aPBX_WT": "E2aPBX 19WT",
    "E2aPBX_19KO": "E2aPBX 19KO",
    "Nalm6_19KO": "Nalm6 19KO",
    "Nalm6_19low": "Nalm6 19Low",
    "Nalm6_19int": "Nalm6 19Int",
    "Nalm6_19hi": "Nalm6 19High"
}

In [ ]:
#@title Receptor levels on OT1 T cells
#celltypes = ['OT1_Naive','OT1_Blast','OT1_CAR']
celltypes = ['temp1','OT1_Naive','OT1_Blast','OT1_CAR']
subsetDf = completeMoleculeNumberDf.query("Cell == @celltypes")
tempDf = pd.DataFrame({'Cell':['temp1','temp1'],'Marker':['TCR','CAR'],'Event':['1','1'],'Molecules (#)':[10,10]}).set_index(['Cell','Marker','Event'])
subsetDf = pd.concat([tempDf,subsetDf])
subsetDf = completeMoleculeNumberDf.query("Cell == @celltypes and (Marker != 'CAR' or Marker == 'CAR' and Cell != 'OT1_Blast')")
subsetDf  = subsetDf.rename(cellTypeRenameDict, level="Cell")
celltypes = list(map(cellTypeRenameDict.get, celltypes))
g = sns.catplot(data=np.log10(subsetDf).reset_index(),x='Marker',y='Molecules (#)',kind='bar',ci='sd',capsize=0.05,errwidth=2,
                hue_order=celltypes,hue='Cell',palette='Purples',linewidth=1,edgecolor='k',aspect=globalAspect, legend=False)
ogyticks = g.axes.flat[0].get_yticks()
newyticks = list(range(0,7))
newyticklabels = ['10$^{'+str(x)+'}$' for x in newyticks]
g.axes.flat[0].set_yticks(newyticks)
g.axes.flat[0].set_yticklabels(newyticklabels)
g.axes.flat[0].set_ylim([2,g.axes.flat[0].get_ylim()[1]])
g.axes.flat[0].set_xlabel('Receptor')
# Clean up legend
handles, labels = g.axes.flat[0].get_legend_handles_labels()
handles2, labels2 = [], []
for h, l in zip(handles, labels):
    if not l.startswith("temp"):
        handles2.append(h)
        labels2.append(l)
g.add_legend(handles=handles2, labels=labels2, title="Cell")
if do_save_plots:
    g.figure.savefig(pj(fig_dir, 'tcellMoleculeNumbers.pdf'),bbox_inches='tight',
              transparent=True, bbox_extra_artists=(g.legend,))
plt.show()
plt.close()

In [ ]:
#@title Ligand levels on B6, B16, E2aPBX cells
celltypes = ['B6','B16','E2aPBX_WT','E2aPBX_19KO']
subsetDf = completeMoleculeNumberDf.query("Cell == @celltypes and (Marker != 'CD19' or Marker == 'CD19' and Cell != 'B6')")
subsetDf  = subsetDf.rename(cellTypeRenameDict, level="Cell")
celltypes = list(map(cellTypeRenameDict.get, celltypes))
g = sns.catplot(data=np.log10(subsetDf).reset_index(),x='Marker',y='Molecules (#)',kind='bar',ci='sd',
                capsize=0.065,errwidth=2,hue_order=celltypes,hue='Cell',order=['MHC','CD19'],palette='Reds',
                linewidth=1,edgecolor='k',aspect=globalAspect)
ogyticks = g.axes.flat[0].get_yticks()
newyticks = list(range(0,7))
newyticklabels = ['10$^{'+str(x)+'}$' for x in newyticks]
g.axes.flat[0].set_yticks(newyticks)
g.axes.flat[0].set_yticklabels(newyticklabels)
g.axes.flat[0].set_xticklabels(['H2Kb','CD19'])
g.axes.flat[0].set_xlabel('Ligand')
g.axes.flat[0].set_ylim([2,g.axes.flat[0].get_ylim()[1]])
if do_save_plots:
    g.figure.savefig(pj(fig_dir, 'mouseAPCMoleculeNumbers.pdf'),bbox_inches='tight',
              transparent=True, bbox_extra_artists=(g.legend,))
plt.show()
plt.close()

In [ ]:
#@title Ligand levels on Nalm6 cells
celltypes = ['Nalm6_19hi','Nalm6_19int','Nalm6_19low','Nalm6_19KO'][::-1]
subsetDf = completeMoleculeNumberDf.query("Cell == @celltypes")
subsetDf  = subsetDf.rename(cellTypeRenameDict, level="Cell")
celltypes = list(map(cellTypeRenameDict.get, celltypes))
g = sns.catplot(data=np.log10(subsetDf).reset_index(),x='Marker',y='Molecules (#)',kind='bar',
                ci='sd',capsize=0.05,errwidth=2,hue_order=celltypes,hue='Cell',order=['MHC','CD19'],
                palette='Greens',linewidth=1,edgecolor='k',aspect=globalAspect)
ogyticks = g.axes.flat[0].get_yticks()
newyticks = list(range(0,7))
newyticklabels = ['10$^{'+str(x)+'}$' for x in newyticks]
g.axes.flat[0].set_yticks(newyticks)
g.axes.flat[0].set_yticklabels(newyticklabels)
g.axes.flat[0].set_xticklabels(['HLA-A2','CD19'])
g.axes.flat[0].set_xlabel('Ligand')
g.axes.flat[0].set_ylim([2,g.axes.flat[0].get_ylim()[1]])
if do_save_plots:
    g.figure.savefig(pj(fig_dir, 'Nalm6MoleculeNumbers.pdf'),bbox_inches='tight',
              transparent=True, bbox_extra_artists=(g.legend,))
plt.show()
plt.close()

In [ ]:
#@title Ligand levels on Baker cells
celltypes = ['temp1','BEAS2B','PC9','temp2']
subsetDf = completeMoleculeNumberDf.query("Cell == @celltypes")
tempDf = pd.DataFrame({'Cell':['temp1','temp2','temp1','temp2'],'Marker':['MHC','MHC','Her2','Her2'],'Event':['1','1','1','1'],'Molecules (#)':[10,10,10,10]}).set_index(['Cell','Marker','Event'])
subsetDf = pd.concat([tempDf,subsetDf])
g = sns.catplot(data=np.log10(subsetDf).reset_index(),x='Marker',y='Molecules (#)',kind='bar',ci='sd',capsize=0.05,
                errwidth=2,hue_order=celltypes,hue='Cell',order=['MHC','Her2'],palette='Blues',linewidth=1,edgecolor='k',
                aspect=globalAspect, legend=False)
ogyticks = g.axes.flat[0].get_yticks()
newyticks = list(range(0,7))
newyticklabels = ['10$^{'+str(x)+'}$' for x in newyticks]
g.axes.flat[0].set_yticks(newyticks)
g.axes.flat[0].set_yticklabels(newyticklabels)
g.axes.flat[0].set_xticklabels(['HLA-A2','Her2'])
g.axes.flat[0].set_xlabel('Ligand')
g.axes.flat[0].set_ylim([2,g.axes.flat[0].get_ylim()[1]])
# Clean up legend
handles, labels = g.axes.flat[0].get_legend_handles_labels()
handles2, labels2 = [], []
for h, l in zip(handles, labels):
    if not l.startswith("temp"):
        handles2.append(h)
        labels2.append(l)
g.add_legend(handles=handles2, labels=labels2, title="Cell")
if do_save_plots:
    g.figure.savefig(pj(fig_dir, 'BakerMoleculeNumbers.pdf'),bbox_inches='tight',
              transparent=True, bbox_extra_artists=(g.legend,))
plt.show()
plt.close()